# Práctica 4

## Ejercicio 1

In [6]:
import numpy as np
a = np.array([1,1,1])
b = np.array([1,2])
c = np.array([a,b], dtype = object)

d = np.outer(a,b)
print(d)

[[1 2]
 [1 2]
 [1 2]]


In [20]:
#Import libraries
import numpy as np
import matplotlib.pyplot as plt


#Def regla XOR
def XOR(x1, x2):
    #x1, x2: +1 o -1
    return x1 * x2

#Def datos x e y
x_data = np.empty([4, 2])
y_data = np.empty(4)

x_data[0] = np.array([1,1])
x_data[1] = np.array([1,-1])
x_data[2] = np.array([-1,1])
x_data[3] = np.array([-1,-1])

for i in range(len(y_data)):
    y_data[i] = XOR(x_data[i][0], x_data[i][1])


#Def función de validación
def validacion():

    pass

#Def función de transferencia
def g(h_vec):

    return np.tanh(h_vec)

def g_prima(h_vec):
    return 1 - g(h_vec)**2

#Def la red neuronal - arquitectura A
# def red_A(input, pesos):
#     '''
#     input: x1 y x2
#     pesos: ndarray de pesos. Cada elemento es una matriz correspondiente a los pesos de una capa a la siguiente

#     En esta arquitectura: pesos = [w1, w2] donde
#         w1: matriz de pesos salientes de la capa 1 y entrantes a la 2 (3x2 en total contando bias)
#         w2: matriz de pesos salientes de la capa 2 (3x1 en total contando bias)
#     '''

#     pass

w_ini_max = 0.1
red_A = {"input":2, "hidden":2, "output":1, "pesos":[np.random.rand(3,2)*w_ini_max, np.random.rand(3,1)*w_ini_max]}


#Def la red neuronal - arquitectura B
# def red_B(input, pesos):
#     '''
#     input: x1 y x2

#     '''

#     pass

#Def algoritmo de retropropagación de errores
def back_propagation(x_data, y_data, red, eta):
    #Se usa la nomenclatura del Hertz

    #Loop sobre las muestras
    for i in range(len(y_data)):   
        #Forward pass
        V_0 = np.concatenate((x_data[i], np.array([-1])), axis = 0) #Agrego el bias
        w_1 = red["pesos"][0]
        h_1 = np.dot(w_1.T, V_0)
        V_1 = np.concatenate((g(h_1), np.array([-1])), axis = 0)

        w_2 = red["pesos"][1]
        h_2 = np.dot(w_2.T, V_1)
        V_2 = g(h_2)

        #Backward
        #Calculo el error de la capa de salida
        delta_2 = g_prima(h_2)*(y_data[i] - V_2)

        #Calculo el error de la capa oculta
        delta_1 = np.concatenate([g_prima(h_1),np.array([1])])*np.dot(w_2, delta_2)

        #Actualizo pesos
        w_1 += eta * np.outer(V_0, delta_1[:2])
        w_2 += eta * np.outer(V_1, delta_2)

        red["pesos"] = [w_1, w_2]

    return red

#Def algoritmo de aprendizaje
def aprendizaje(x_data, y_data, red, eta, epochs = 1):
    #Loop sobre las epochs
    for i in range(epochs):
        #Backpropagation
        red = back_propagation(x_data, y_data, red, eta)
    
    return red

#Entreno red A
eta = 0.1
model_A = aprendizaje(x_data, y_data, red_A, eta, epochs = 100)

#Entreno red B

#Grafico